In [1]:
import requests
from bs4 import BeautifulSoup

In [258]:
# 最初に取ってくる要素(名前、リンク)を格納するリストを作成
tintai_names = []
tintai_links = []

# 13ページ分処理を繰り返す
for page in range(1,14):
    REQUEST_URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=030&bs=040&fw2=&pc=10&po1=25&po2=99&ta=14&sc=14102&md=01&md=02&ts=1&cb=0.0&ct=7.0&et=10&mb=0&mt=9999999&cn=9999999&tc=0400101&tc=0400501&tc=0400502&tc=0400601&tc=0400301&tc=0400303&shkr1=03&shkr2=03&shkr3=03&shkr4=03&shkk1=02060202&page={}'.format(page) 
    res = requests.get(REQUEST_URL)
    result =res.text
    soup = BeautifulSoup(res.text,'html.parser')

    # 賃貸リストのタイトルとURLが記述されたclass=js-cassetLinkHrefを全部取ってくる
    elems = soup.select(".js-cassetLinkHref")
    
    # elemsに格納された要素1つずつからタイトル名部分とリンク部分を抽出し、それぞれのリストに追加
    for elem in elems:
        tintai_name = elem.text.replace("[","").replace("]","")
        tintai_link = elem.attrs['href']
        tintai_names.append(tintai_name)
        tintai_links.append(tintai_link)

# 今回の検索条件結果ページ124件だったので全部とれてる
print(len(tintai_names))
    # print(len(tintai_links))

102


In [259]:
import pandas as pd
df = pd.DataFrame()
df["物件名"] = tintai_names
df["リンク"] = tintai_links
df

,物件名,リンク
0,リヴシティ横浜東ベイサイド,/chintai/bc_100343313667/
1,リヴシティ横浜東ベイサイド,/chintai/bc_100343314105/
2,リヴシティ横浜東ベイサイド,/chintai/bc_100343313391/
3,マリーナヨコハマベイサイド 203号室,/chintai/bc_100343842510/
4,マリーナヨコハマベイサイド 203号室,/chintai/bc_100343709215/
...,...,...
97,リヴシティ横濱七島町 604号室,/chintai/bc_100336530692/
98,京急本線 子安駅 7階建 築7年,/chintai/bc_100341277604/
99,京急本線 子安駅 7階建 築7年,/chintai/bc_100333151536/
100,京急本線 子安駅 7階建 築7年,/chintai/bc_100342297202/


In [260]:
df.to_csv('物件情報.csv',index=False)

取得したURLから'部屋の特徴・設備'を取得

In [8]:
# 情報取得方法を試すためデータを3行分に絞る
df_test = df.iloc[[0,4,10],:]
df_test

,物件名,リンク
0,リヴシティ横浜東ベイサイド,/chintai/bc_100343313667/
4,マリーナヨコハマベイサイド 203号室,/chintai/bc_100343709215/
10,クレヴィスタ横浜新子安,/chintai/bc_100342548057/


In [261]:
import re

property_infs = []
features_infs = []
yen_infs = []
access_infs = []
env_infs = []

for bukken_url in df["リンク"]:
    url = "https://suumo.jp" + bukken_url
    print(url)
    
    res_2 = requests.get(url)
    result_2 =res.text
    soup_2 = BeautifulSoup(res_2.text,'html.parser')


    # 部屋情報リスト
    floor_type = soup_2.select_one(".property_view_detail--floor_type").select("li")
    # 概要リスト
    overview = soup_2.select_one(".table_gaiyou").select("td")

    # suumo物件コード
    # propertyCd = []
    property_cd = overview[10].text
    # propertyCd.append(property_cd)

    # 物件基本情報
    property_inf = []
    # 物件名
    property_name = soup_2.select_one(".section_h1-header-title").get_text(strip=True)
    # 所在地
    location= soup_2.select_one(".property_view_detail--location").select_one(".property_view_detail-text").get_text(strip=True)
    # 築年月
    construction_date = overview[3].get_text(strip=True)
    # 建築種別
    building_type = floor_type[3].select_one(".property_data-body").get_text(strip=True)
    # 構造
    structure = overview[1].get_text(strip=True)
    # 向き
    direction = floor_type[2].select_one(".property_data-body").get_text(strip=True)
    #階建
    number_of_floor = overview[2].get_text(strip=True)
    # 間取り
    floor_plan = floor_type[0].select_one(".property_data-body").get_text(strip=True)
    #間取り詳細
    floor_plan_detail = overview[0].get_text(strip=True)
    #専有面積
    area = floor_type[1].select_one(".property_data-body").get_text(strip=True)
    # 間取り図
    area_img = soup_2.select_one("img[alt='間取り図']").get('data-src')
    # 取引態様
    transaction_mode = overview[7].get_text(strip=True)
    # 取引先
    shop_name = soup_2.select_one("p.shop-name.fl").select_one("a").get_text(strip=True)
    # 更新日
    update_date = overview[12].get_text(strip=True)

    property_inf.append(property_cd)
    property_inf.append(property_name)
    property_inf.append(location)
    property_inf.append(construction_date)
    property_inf.append(building_type)
    property_inf.append(structure)
    property_inf.append(direction)
    property_inf.append(number_of_floor)
    property_inf.append(floor_plan)
    property_inf.append(floor_plan_detail)
    property_inf.append(area)
    property_inf.append(area_img)
    property_inf.append(transaction_mode)
    property_inf.append(shop_name)
    property_inf.append(update_date)

    property_infs.append(property_inf)


    # 設備情報
    features_inf = []
    features = soup_2.select_one("#bkdt-option").select_one("li").get_text(strip=True)

    features_inf.append(property_cd)
    features_inf.append(features)
    features_infs.append(features_inf)

    #基本費用　その他は除く
    # 費用リスト
    yen_inf = []
    yen = soup_2.select_one(".property_view_detail--yen")
    sub = yen.select_one(".property_view_detail-info-sub").select("li")

    # 賃料
    rent = yen.select_one(".property_view_main-emphasis").get_text(strip=True)
    # 管理費
    management_fee = yen.select_one(".property_data--main").select_one(".property_data-body").get_text(strip=True)
    # 敷金/礼金
    security_deposit_key_money = sub[0].select("span")
    security_deposit = security_deposit_key_money[0].get_text(strip=True)
    key_money = security_deposit_key_money[2].get_text(strip=True)
    # 保証金
    deposit = sub[1].select_one(".property_data-body").get_text(strip=True)
    # 敷引・償却
    depreciation = sub[2].select_one(".property_data-body").get_text(strip=True)

    yen_inf.append(property_cd)
    yen_inf.append(rent)
    yen_inf.append(management_fee)
    yen_inf.append(security_deposit)
    yen_inf.append(key_money)
    yen_inf.append(deposit)
    yen_inf.append(depreciation)

    yen_infs.append(yen_inf)


    # アクセス
    accesses = soup_2.select_one(".property_view_detail--train")
    accesses = accesses.select(".property_view_detail-text")

    for elem in accesses:
        access_inf=[]
        access_data = elem.get_text(strip=True)
        space_idx = access_data.find(' ')
        station_name = access_data[:space_idx]
        walk_time = access_data[space_idx+1:]
        access_inf.append(property_cd)
        access_inf.append(station_name)
        access_inf.append(walk_time)
        access_infs.append(access_inf)


    # 周辺環境
    # 概要の情報がないものがありoverview[19]が指定できないときがある
    # そのため、周辺情報は概要全体のテキストを抽出した後、文字列指定で抽出、li毎の分割
    # surrounding_environments = overview[19].select("li")
    surrounding_environments = soup_2.select_one(".table_gaiyou").get_text(strip=True)

    surrounding_environments_idx = surrounding_environments.find('周辺情報')
    surrounding_environments = surrounding_environments[surrounding_environments_idx+4:]
    QRcode_idx = surrounding_environments.find('携帯用QRコード')
    surrounding_environments = surrounding_environments[:QRcode_idx]
    # mを基準に分割するとmが消えるが、下記方法で分割すると消えない
    surrounding_environments = re.split(r'(?<=m)',surrounding_environments)

    # 施設名・施設分類・距離に分割
    for elem in surrounding_environments:
        env_data = elem
        env_inf = []

        building_name_idx = env_data.find('（')
        building_name = env_data[:building_name_idx]

        other_data = env_data[building_name_idx+1:]
        building_ganre_idx = other_data.find('）')
        building_ganre = other_data[:building_ganre_idx]

        distance_idx = other_data.find('で')
        distance = other_data[distance_idx+1:]

        env_inf.append(property_cd)
        env_inf.append(building_name)
        env_inf.append(building_ganre)
        env_inf.append(distance)

        env_infs.append(env_inf)
    


# DF変換
property_t = pd.DataFrame(property_infs)
property_t.columns=['物件CD','物件名','所在地','築年数','建築種別','構造','向き','階建','間取り','間取り詳細','専有面積','間取り図','取引様態','取引先','更新日']

option_t = pd.DataFrame(features_infs)
option_t.columns=['物件CD','オプション']

cost_t=pd.DataFrame(yen_infs)
cost_t.columns=['物件CD','賃料','管理費','敷金','礼金','保証金','敷引・償却']

access_t=pd.DataFrame(access_infs)
access_t.columns=['物件CD','駅','徒歩分数']

environment_t = pd.DataFrame(env_infs)
environment_t.columns=['物件CD','施設名','ジャンル','距離']



https://suumo.jp/chintai/bc_100343313667/
https://suumo.jp/chintai/bc_100343314105/
https://suumo.jp/chintai/bc_100343313391/
https://suumo.jp/chintai/bc_100343842510/
https://suumo.jp/chintai/bc_100343709215/
https://suumo.jp/chintai/bc_100342823375/
https://suumo.jp/chintai/bc_100339853853/
https://suumo.jp/chintai/bc_100339856694/
https://suumo.jp/chintai/bc_100341719751/
https://suumo.jp/chintai/bc_100342503124/
https://suumo.jp/chintai/bc_100342548057/
https://suumo.jp/chintai/bc_100340883623/
https://suumo.jp/chintai/bc_100342137982/
https://suumo.jp/chintai/bc_100342028513/
https://suumo.jp/chintai/bc_100344058269/
https://suumo.jp/chintai/bc_100316093085/
https://suumo.jp/chintai/bc_100335872435/
https://suumo.jp/chintai/bc_100330020699/
https://suumo.jp/chintai/bc_100330021137/
https://suumo.jp/chintai/bc_100341720649/
https://suumo.jp/chintai/bc_100339757342/
https://suumo.jp/chintai/bc_100334889538/
https://suumo.jp/chintai/bc_100343719318/
https://suumo.jp/chintai/bc_100343

In [262]:
property_t.to_csv('物件基本情報.csv',index=False)
option_t.to_csv('設備情報.csv',index=False)
cost_t.to_csv('費用情報.csv',index=False)
access_t.to_csv('アクセス.csv',index=False)
environment_t.to_csv('周辺環境.csv',index=False)


In [79]:
# URL = "https://suumo.jp/chintai/bc_100342823375/"
# res_2 = requests.get(URL)
# result_2 = res_2.text
# soup_2 = BeautifulSoup(res_2.text,'html.parser')

<a href="/chintai/kaisha/kc_030_169274002/" target="_blank"><img alt="" height="11" src="/jj/jjcommon/img/icon-shopname_s.gif" width="16">アパマンショップ関内店(株)Select</img></a>

In [255]:

# # 部屋情報リスト
# floor_type = soup_2.select_one(".property_view_detail--floor_type").select("li")

# # 概要リスト
# overview = soup_2.select_one(".table_gaiyou").select("td")

# # suumo物件コード
# # propertyCd = []
# property_cd = overview[10].text
# # propertyCd.append(property_cd)

# # 物件基本情報
# property_infs = []
# property_inf = []
# # 物件名
# property_name = soup_2.select_one(".section_h1-header-title").get_text(strip=True)

# # 所在地
# location= soup_2.select_one(".property_view_detail--location").select_one(".property_view_detail-text").get_text(strip=True)

# # 築年月
# construction_date = overview[3].get_text(strip=True)

# # 建築種別
# building_type = floor_type[3].select_one(".property_data-body").get_text(strip=True)

# # 構造
# structure = overview[1].get_text(strip=True)

# # 向き
# direction = floor_type[2].select_one(".property_data-body").get_text(strip=True)

# #階建
# number_of_floor = overview[2].get_text(strip=True)

# # 間取り
# floor_plan = floor_type[0].select_one(".property_data-body").get_text(strip=True)

# #間取り詳細
# floor_plan_detail = overview[0].get_text(strip=True)

# #専有面積
# area = floor_type[1].select_one(".property_data-body").get_text(strip=True)

# # 間取り図
# area_img = soup_2.select_one("img[alt='間取り図']").get('data-src')

# # 取引態様
# transaction_mode = overview[7].get_text(strip=True)

# # 取引先
# shop_name = soup_2.select_one("p.shop-name.fl").select_one("a").get_text(strip=True)

# # 更新日
# update_date = overview[12].get_text(strip=True)

# property_inf.append(property_cd)
# property_inf.append(property_name)
# property_inf.append(location)
# property_inf.append(construction_date)
# property_inf.append(building_type)
# property_inf.append(structure)
# property_inf.append(direction)
# property_inf.append(number_of_floor)
# property_inf.append(floor_plan)
# property_inf.append(floor_plan_detail)
# property_inf.append(area)
# property_inf.append(area_img)
# property_inf.append(transaction_mode)
# property_inf.append(shop_name)
# property_inf.append(update_date)

# property_infs.append(property_inf)

# # 設備情報
# features = soup_2.select_one("#bkdt-option").select_one("li").get_text(strip=True)


# #基本費用　その他は除く
# # 費用リスト
# yen = soup_2.select_one(".property_view_detail--yen")
# sub = yen.select_one(".property_view_detail-info-sub").select("li")

# # 賃料
# rent = yen.select_one(".property_view_main-emphasis").get_text(strip=True)

# # 管理費
# management_fee = yen.select_one(".property_data--main").select_one(".property_data-body").get_text(strip=True)

# # 敷金/礼金
# security_deposit_key_money = sub[0].select("span")
# security_deposit = security_deposit_key_money[0].get_text(strip=True)
# key_money = security_deposit_key_money[2].get_text(strip=True)

# # 保証金
# deposit = sub[1].select_one(".property_data-body").get_text(strip=True)

# # 敷引・償却
# depreciation = sub[2].select_one(".property_data-body").get_text(strip=True)

# # アクセス
# accesses = soup_2.select_one(".property_view_detail--train")
# accesses = accesses.select(".property_view_detail-text")
# access = []
# for elem in accesses:
#     access_inf=[]
#     access_data = elem.get_text(strip=True)
#     space_idx = access_data.find(' ')
#     station_name = access_data[:space_idx]
#     walk_time = access_data[space_idx+1:]
#     access_inf.append(property_cd)
#     access_inf.append(station_name)
#     access_inf.append(walk_time)
# #     access.append(access_inf)
# import re
# # # 周辺環境
# surrounding_environments = soup_2.select_one(".table_gaiyou").get_text(strip=True)

# surrounding_environments_idx = surrounding_environments.find('周辺情報')
# surrounding_environments = surrounding_environments[surrounding_environments_idx+4:]
# QRcode_idx = surrounding_environments.find('携帯用QRコード')
# surrounding_environments = surrounding_environments[:QRcode_idx]
# surrounding_environments = re.split(r'(?<=m)',surrounding_environments)


#     # 施設名・施設分類・距離に分割
# surrounding_environment = []
# for elem in surrounding_environments:
#     env_data = elem.get_text(strip=True)
#     eachEnv = []
    
#     building_name_idx = env_data.find('（')
#     building_name = env_data[:building_name_idx]
#     eachEnv.append(building_name)

#     other_data = env_data[building_name_idx+1:]
#     building_ganre_idx = other_data.find('）')
#     building_ganre = other_data[:building_ganre_idx]
#     eachEnv.append(building_ganre)

#     distance_idx = other_data.find('で')
#     distance = other_data[distance_idx+1:]
#     eachEnv.append(distance)

#     surrounding_environment.append(eachEnv)

# access

['オルトヨコハマ（ショッピングセンター）まで620m', 'そうてつローゼン大口店（スーパー）まで569m', '横濱屋大口店（スーパー）まで468m', 'セブンイレブン横浜大口通店（コンビニ）まで308m', 'ローソン横浜大口通店（コンビニ）まで391m', 'くすりセイジョー大口店（ドラッグストア）まで558m', '']


取得した情報をテーブル単位でDF作成、
設備情報は条件が含まれているか否かでフラグを持つように編集

In [170]:
# import pandas as pd

In [213]:
# property_m = pd.DataFrame(property_infs)
# # property_m['物件CD']=propertyCd
# # property_m['物件名']=property_name
# # property_m['所在地']=location
# # property_m['築年数']=construction_date
# # property_m['建築種別']=building_type
# # property_m['構造']=structure
# # property_m['向き']=direction
# # property_m['階健']=number_of_floor
# # property_m['間取り']=floor_plan
# # property_m['間取り詳細']=floor_plan_detail
# # property_m['専有面積']=area
# # property_m['間取り図']=area_img
# # property_m['取引態様']=transaction_mode
# # property_m['取引先']=shop_name
# # property_m['更新日']=update_date

# environment_t = pd.DataFrame(surrounding_environment)
# environment_t.columns=['施設名','ジャンル','距離']

# cost_t=pd.DataFrame()
# cost_t['物件CD']=propertyCd
# cost_t['賃料']=rent
# cost_t['管理費']=management_fee
# cost_t['敷金']=security_deposit
# cost_t['礼金']=key_money
# cost_t['保証金']=deposit
# cost_t['敷引・償却']=depreciation

# access_t=pd.DataFrame(access)
# access_t.columns=['物件CD','駅','徒歩分数']

# option_t = pd.DataFrame()
# option_t['物件ID']=propertyCd
# option_t['オプション']=features

# # print(access_t)
# print(property_m)

             0                                                1    
0  100342823375  リヴシティ横浜東ベイサイド - アパマンショップ関内店(株)Selectが提供する賃貸物件情報  \

                 2         3      4     5   6        7   8       9        10   
0  神奈川県横浜市神奈川区東神奈川２  2010年11月  マンション  鉄筋コン  南東  6階/10階建  1K  洋6.7 K  20.02m2  \

                                                  11  12   
0  https://img01.suumo.com/front/gazo/fr/bukken/3...  仲介  \

                     13          14  
0  アパマンショップ関内店(株)Select  2023/09/12  
